In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
import os
os.chdir('..')

# Target transformations
> Seamlessly transform target values

Since mlforecast uses a single global model it can be helpful to apply some transformations to the target to ensure that all series have similar distributions, they can also help remove trend for models that can't deal with it out of the box.

### Local transformations
> Transformations applied per serie

### Data setup

For this example we'll use a single serie from the M4 dataset.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datasetsforecast.m4 import M4
from sklearn.base import BaseEstimator

from mlforecast import MLForecast
from mlforecast.target_transforms import Differences

In [ ]:
data_path = 'data'
await M4.async_download(data_path, group='Hourly')
df, *_ = M4.load(data_path, 'Hourly')
df['ds'] = df['ds'].astype('int32')
serie = df[df['unique_id'].eq('H196')]

### Differences
We'll take a look at our serie to see possible differences that would help our models.

In [ ]:
def plot(series, fname):
    n_series = len(series)
    fig, ax = plt.subplots(ncols=n_series, figsize=(7 * n_series, 6), squeeze=False)
    for (title, serie), axi in zip(series.items(), ax.flat):
        serie.set_index('ds')['y'].plot(title=title, ax=axi)
    fig.savefig(fname, bbox_inches='tight')
    plt.close()

In [ ]:
plot({'original': serie}, 'figs/target_transforms__eda.png')

![](../figs/target_transforms__eda.png)

We can see that our data has a trend as well as a clear seasonality. We can try removing the trend first.

In [ ]:
diff1 = Differences([1])
diff1.set_column_names('unique_id', 'ds', 'y')  # this is done for you when using MLForecast
without_trend = diff1.fit_transform(serie)
plot({'original': serie, 'without trend': without_trend}, 'figs/target_transforms__diff1.png')

![](../figs/target_transforms__diff1.png)

The trend is gone, we can now try taking the 24 difference (subtract the value at the same hour in the previous day).

In [ ]:
diff2 = Differences([1, 24])
diff2.set_column_names('unique_id', 'ds', 'y')
without_trend_and_seasonality = diff2.fit_transform(serie)
plot({'original': serie, 'without trend and seasonality': without_trend_and_seasonality}, 'figs/target_transforms__diff2.png')

![](../figs/target_transforms__diff2.png)

We see that our serie is random noise now, so we could try forecasting it with a model that always predicts 0, which will basically project the trend and seasonality.

In [ ]:
class Zeros(BaseEstimator):
    def fit(self, X, y=None):
        return self

    def predict(self, X, y=None):
        return np.zeros(X.shape[0])

fcst = MLForecast(
    models=Zeros(),
    freq=1,
    target_transforms=[Differences([1, 24])],
)
preds = fcst.fit(serie).predict(48)
fig, ax = plt.subplots()
pd.concat([serie.tail(24 * 10), preds]).set_index('ds').plot(ax=ax)
plt.close()
fig.savefig('figs/target_transforms__zeros.png')

![](../figs/target_transforms__zeros.png)